In [2]:
import os
import json
import re
import jieba

from opencc import OpenCC
from os import listdir
from gensim.models import word2vec

def sc_to_tc(string): #簡體中文轉繁體中文
    cc = OpenCC('s2t') 
    return cc.convert(string)

def remain_language(string): #留下文字
    result = re.sub('\W+', ' ', string).replace("_",  '')
    return result

def transform_and_write_data(file_path): #把wiki_zh濃縮成一個.txt
    files_layer_1 = listdir(file_path)
    #print(files_layer_1)
    
    with open( "./wiki_zh_seq.txt" , 'w', encoding='utf-8') as new_file: 
        for file_layer_1 in files_layer_1:
            if file_layer_1=='.DS_Store':
                continue
            else:
                files_layer_2 = listdir(file_path+"/"+file_layer_1)
                #print(files_layer_2)
                
                for file_layer_2 in files_layer_2:
                    with open( file_path+"/"+file_layer_1+"/"+file_layer_2 , 'r', encoding='utf-8') as file: #開wiki_zh的每個txt檔
                        for times, data in enumerate(file, 1):
                            #print('data num:', times)
                            data = json.loads(data)
                            data = data['text']
                            data = remain_language(data)
                            data = sc_to_tc(data) #轉繁體中文
                            data = jieba.lcut(data) 
                            data = [word for word in data if word != ' ']
                            data = ' '.join(data)+'\n'

                            new_file.write(data)


/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [26]:
def training(train_data_path): #訓練並儲存模型
    
    seed = 666 #亂數種子
    sg = 0 #用CBOW(0)或是Skip-gram(1)
    window_size = 10 #周圍詞彙要看多少範圍
    vector_size = 100 #轉成向量的維度
    min_count = 1 #詞頻少於 min_count 之詞彙不會參與訓練
    workers = 8 #訓練的並行數量
    epochs = 5 #訓練的迭代次數
    batch_words = 10000 #每次給予多少詞彙量訓練

    train_data = word2vec.LineSentence(train_data_path)
    
    model = word2vec.Word2Vec(
        train_data,
        min_count=min_count,
        vector_size=vector_size,
        workers=workers,
        epochs=epochs,
        window=window_size,
        sg=sg,
        seed=seed,
        batch_words=batch_words
    )

    model.save('word2vec.model') #儲存模型


In [32]:
file_path="./wiki_zh"
train_data_path="./wiki_zh_seq.txt"

transform_and_write_data(file_path) #檔案整理,並輸出

training(train_data_path) #訓練模型

model = word2vec.Word2Vec.load('word2vec.model')

In [43]:
print("請輸入一個物件")
ITME = input()

print()
print(ITME,"的 top 20 相關詞")
for item in model.wv.most_similar(ITME, topn=20):
    print(item)

請輸入一個物件
水果

水果 的 top 20 相關詞
('蔬菜', 0.887900173664093)
('蜂蜜', 0.8488485813140869)
('柑橘', 0.8451145887374878)
('蔬果', 0.8251779079437256)
('豆類', 0.8211118578910828)
('葡萄乾', 0.8155094385147095)
('芋頭', 0.8050132393836975)
('覆盆子', 0.8043386936187744)
('橄欖油', 0.8028526902198792)
('玉米', 0.8016112446784973)
('薰衣草', 0.7992830276489258)
('大蒜', 0.7992480993270874)
('檸檬', 0.7978089451789856)
('辣椒', 0.7973421812057495)
('黑胡椒', 0.79655921459198)
('香草', 0.7941129803657532)
('藍莓', 0.7931357026100159)
('雞肉', 0.7906715869903564)
('鳳梨', 0.7893867492675781)
('椰子', 0.7886860370635986)
